In [85]:
import itertools

import numpy
from sympy import init_session

In [ ]:
init_session()

In [99]:
c = symbols("c0:4")
s = symbols("s0:4")
a = symbols("a0:4")
b = symbols("b0:4")

In [104]:
def dot(es1, es2):
    return sum([e[0]*e[1] for e in zip(es1, es2)])

In [67]:
# Why doesn't sympy do this by itself? Probably some maths thing…
def definize(F, s, a, b):
    return F.subs(s, b) - F.subs(s, a)

In [71]:
def do_integral(f, (s, a, b)):
    i = Integral(f, s)
    return definize(i.doit(), s, a, b)

In [101]:
da_integral = reduce(do_integral, zip(s, a, b)[1:], cos(pi * dot(s, c)))

In [81]:
# Copied from OffSwitchCartpole.ipynb.
clipped_high = np.array([2.5, 3.6, 0.28, 3.7])
clipped_low  = -clipped_high
state_ranges = np.array([clipped_low, clipped_high])


In [102]:
bounds_int = da_integral.subs(zip(a[1:], clipped_low[1:])).subs(zip(b[1:], clipped_high[1:]))

In [88]:
# Copied from hiora_cartpole.fourier_fa.
order = 3
n_dims = 4
c_matrix = np.array(
               list( itertools.product(range(order+1), repeat=n_dims) ),
               dtype=np.int32)

In [89]:
def sum_term(integral, c, c_vec):
    return integral.subs(zip(c, c_vec))

In [117]:
sum_terms = [sum_term(bounds_int, c, c_vec) for c_vec in c_matrix]

In [120]:
np_sum_terms = [lambdify(s[0], sum_term, 'numpy') for sum_term in sum_terms]

In [126]:
def phi(np_sum_terms, theta, s0):
    return np.dot(theta, 
                  np.array([npst(s0) for npst in np_sum_terms]))